In [10]:
import pandas as pd
from pulp import *

In [11]:
df_distance = pd.read_excel('../../Datos_P1/df_distance_km.xlsx')
df_location = pd.read_excel('../../Datos_P1/df_location.xlsx')
df_orders = pd.read_excel('../../Datos_P1/df_orders.xlsx')
df_vehicle = pd.read_excel('../../Datos_P1/df_vehicle.xlsx')

df_distance.index = df_distance.columns

In [12]:
df_vehicle

,vehiculo_id,capacidad_kg,costo_km,autonomia_km
0,1,2026,0.20,603
1,2,4362,0.14,630
2,3,4881,0.20,664
3,4,3321,0.19,514
4,5,10000,0.32,350
5,6,3129,0.14,791


In [13]:
df_vehicle.sort_values(by='capacidad_kg', ascending=False, inplace=True) #Ordenar los vehiculos por capacidad, de mayor a menor
df_vehicle

,vehiculo_id,capacidad_kg,costo_km,autonomia_km
4,5,10000,0.32,350
2,3,4881,0.20,664
1,2,4362,0.14,630
3,4,3321,0.19,514
5,6,3129,0.14,791
0,1,2026,0.20,603


In [14]:
df_unico_vehiculo = df_vehicle.iloc[[0]] #Hacer un dataFrame nuevo solo con el primer dato del otro dataframe (el vehiculo con más capacidad)
df_unico_vehiculo.reset_index(drop=True, inplace=True) #Resetear el index del nuevo dataframe para que no tenga indice 4
df_unico_vehiculo

,vehiculo_id,capacidad_kg,costo_km,autonomia_km
0,5,10000,0.32,350


In [15]:
df_unico_vehiculo.at[0, 'vehiculo_id'] = 1 #Cambio del vehiculo_id para que sea más agradable trabajar con ello
df_unico_vehiculo.at[0, 'costo_km'] = 0.1
df_unico_vehiculo.at[0, 'autonomia_km'] = 1000
df_unico_vehiculo.at[0, 'capacidad_kg'] = 30000
df_vehicle = df_unico_vehiculo

Haz los cambios que veas a la capacidad y demás datos, si no quieres que tenga el vehiculo_id de 1 quita lo de antes y ya


In [16]:
# Preparación datos para PuLP
capacidades = dict(zip(df_vehicle["vehiculo_id"], df_vehicle["capacidad_kg"]))
costos = dict(zip(df_vehicle["vehiculo_id"], df_vehicle["costo_km"]))
autonomias = dict(zip(df_vehicle["vehiculo_id"], df_vehicle["autonomia_km"]))

pedidos = dict(zip(df_orders["cliente"], df_orders["order_demand"]))
pedidos["Almacén"] = 0  # Aseguramos que el almacén no tenga demanda

distancias = df_distance.stack().to_dict()  # Convierte a un diccionario (i, j): distancia

clientes = df_orders["cliente"].tolist() + ["Almacén"]  # Incluir el almacén como un cliente especial
vehiculos = df_vehicle["vehiculo_id"].tolist()

In [17]:
# Definir el problema de optimización
problem = LpProblem("Ruteo_de_Vehiculos_Caso4", LpMinimize)

# Listas de clientes y vehículos
clientes = df_orders["cliente"].tolist()
vehiculos = df_vehicle["vehiculo_id"].tolist()
almacen = "Almacén"

# Crear variables de decisión
x = {(v, i, j): LpVariable(f"x_{v}_{i}_{j}", cat="Binary") 
     for v in vehiculos for i in [almacen] + clientes for j in clientes + [almacen] if i != j and df_distance.at[i, j] > 0}

y = {(v, i): LpVariable(f"y_{v}_{i}", cat="Binary") 
     for v in vehiculos for i in clientes}

# Variables auxiliares para evitar subciclos
u = {i: LpVariable(f"u_{i}", lowBound=0, upBound=len(clientes), cat="Continuous") for i in clientes}

# Función objetivo: minimizar costo total del recorrido
problem += lpSum(x[v, i, j] * df_distance.at[i, j] * df_vehicle[df_vehicle["vehiculo_id"] == v]["costo_km"].values[0] 
                 for v in vehiculos for i in [almacen] + clientes for j in clientes + [almacen] if i != j and df_distance.at[i, j] > 0)

# Restricción: Cada cliente debe ser visitado exactamente una vez
for i in clientes:
    problem += lpSum(y[v, i] for v in vehiculos) == 1

# Restricción: Capacidad del vehículo
for v in vehiculos:
    problem += lpSum(y[v, i] * df_orders[df_orders["cliente"] == i]["order_demand"].values[0] 
                     for i in clientes) <= df_vehicle[df_vehicle["vehiculo_id"] == v]["capacidad_kg"].values[0]

# Restricción: Autonomía del vehículo
for v in vehiculos:
    problem += lpSum(x[v, i, j] * df_distance.at[i, j] 
                     for i in [almacen] + clientes for j in clientes + [almacen] if i != j and df_distance.at[i, j] > 0) <= df_vehicle[df_vehicle["vehiculo_id"] == v]["autonomia_km"].values[0]

# Restricción: Salida y llegada desde el almacén
for v in vehiculos:
    problem += lpSum(x[v, almacen, j] for j in clientes if df_distance.at[almacen, j] > 0) == 1
    problem += lpSum(x[v, i, almacen] for i in clientes if df_distance.at[i, almacen] > 0) == 1

# Restricción: Si un vehículo visita un cliente, debe salir de él
for v in vehiculos:
    for i in clientes:
        problem += lpSum(x[v, i, j] for j in clientes + [almacen] if i != j and df_distance.at[i, j] > 0) == y[v, i]
        problem += lpSum(x[v, j, i] for j in [almacen] + clientes if i != j and df_distance.at[j, i] > 0) == y[v, i]

# Restricción para evitar subciclos
for i in clientes:
    for j in clientes:
        if i != j and df_distance.at[i, j] > 0:
            problem += u[i] - u[j] + (len(clientes) * lpSum(x[v, i, j] for v in vehiculos)) <= len(clientes) - 1

# Resolver el problema con un límite de tiempo
problem.solve(PULP_CBC_CMD(timeLimit=60))

1

In [18]:
# Calcular costos, peso total y capacidad utilizada
costo_total = 0
print("\nResumen de rutas:\n" + "="*40)
for v in vehiculos:
    costo_vehiculo = 0
    peso_total = 0
    print(f"\n🚛 Vehículo {v}:")
    for i in [almacen] + clientes:
        for j in clientes + [almacen]:
            if i != j and df_distance.at[i, j] > 0 and x[v, i, j].varValue == 1:
                distancia = df_distance.at[i, j]
                costo_km = df_vehicle[df_vehicle["vehiculo_id"] == v]["costo_km"].values[0]
                costo_vehiculo += distancia * costo_km
                if i != almacen:
                    peso_total += df_orders[df_orders["cliente"] == i]["order_demand"].values[0]
                print(f"   - De {i} a {j}: {distancia} km, Costo: ${distancia * costo_km:.2f}")
    capacidad = df_vehicle[df_vehicle["vehiculo_id"] == v]["capacidad_kg"].values[0]
    print(f"   🔹 Peso total transportado: {peso_total} kg / Capacidad: {capacidad} kg")
    print(f"   🔹 Costo total del vehículo: ${costo_vehiculo:.2f}")
    costo_total += costo_vehiculo

print("\n💰 Costo total de todas las rutas: ${:.2f}".format(costo_total))


Resumen de rutas:

🚛 Vehículo 1:
   - De Almacén a Cliente_20: 14.8282 km, Costo: $1.48
   - De Cliente_1 a Cliente_16: 1.5775 km, Costo: $0.16
   - De Cliente_2 a Cliente_13: 2.6961 km, Costo: $0.27
   - De Cliente_3 a Cliente_19: 7.7574 km, Costo: $0.78
   - De Cliente_4 a Cliente_6: 0.9118999999999999 km, Costo: $0.09
   - De Cliente_5 a Cliente_11: 2.6253 km, Costo: $0.26
   - De Cliente_6 a Cliente_9: 2.906 km, Costo: $0.29
   - De Cliente_7 a Cliente_2: 0.4847 km, Costo: $0.05
   - De Cliente_8 a Cliente_15: 5.9742 km, Costo: $0.60
   - De Cliente_9 a Almacén: 1.0494 km, Costo: $0.10
   - De Cliente_10 a Cliente_17: 10.8948 km, Costo: $1.09
   - De Cliente_11 a Cliente_14: 1.6099 km, Costo: $0.16
   - De Cliente_12 a Cliente_10: 2.3061 km, Costo: $0.23
   - De Cliente_13 a Cliente_18: 6.286899999999999 km, Costo: $0.63
   - De Cliente_14 a Cliente_1: 2.2133 km, Costo: $0.22
   - De Cliente_15 a Cliente_12: 5.6418 km, Costo: $0.56
   - De Cliente_16 a Cliente_4: 1.3127 km, Costo: